### 数据集
- 我们需要以理想的格式生成数据集，以便于两个预训练任务：遮蔽语言模型和下一句预测。
- 一方面，最初的BERT模型是在两个庞大的图书语料库和英语维基百科的合集上预训练的，但它很难吸引这本书的大多数读者。
- 另一方面，现成的预训练BERT模型可能不适合医学等特定领域的应用。因此，在定制的数据集上对BERT进行预训练变得越来越流行。
- 为了方便BERT预训练的演示，我们使用了较小的语料库WikiText-2。
- 与用于预训练word2vec的PTB数据集相比，WikiText-2（1）保留了原来的标点符号，适合于下一句预测；（2）保留了原来的大小写和数字；（3）大了一倍以上。

- 在WikiText-2数据集中，每行代表一个段落，其中在任意标点符号及其前面的词元之间插入空格。
- 保留至少有两句话的段落。为了简单起见，我们仅使用句号作为分隔符来拆分句子。

In [1]:
import random
import torch
from d2l import torch as d2l

In [2]:
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

可以看出文本的加载形式，文本被加载成列表的形式，每个列表又是一个储存了多个句子的列表.
里面的为止部分已经替换成了unk

In [3]:
data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
paragraphs = _read_wiki(data_dir)
paragraphs[2]

['ono , an inhabitant of <unk> ( says an ancient japanese legend of <unk> 545 ) , spent the seasons longing for his ideal of female beauty',
 'he met her one evening on a vast moor and married her',
 "simultaneously with the birth of their son , ono 's dog was delivered of a <unk> which as it grew up became more and more hostile to the lady of the <unk>",
 'she <unk> her husband to kill it , but he refused',
 'at last one day the dog attacked her so furiously that she lost courage , resumed <unk> shape , <unk> over a fence and fled .']

接下来是bert所需的输入数据：
- bert和以往模型不一样的地方在于，有非常多的输入数据
- 1、和其他模型一样，语料库相关数据，
- 2、segment，对于句子对进行前后句子区分
- 3、句子有效长度
- 4、掩码矩阵权重、掩码信息
- 5、下一个句子预测训练样本

先为BERT的两个预训练任务实现辅助函数。这些辅助函数将在稍后将原始文本语料库转换为理想格式的数据集时调用，以预训练BERT。

_get_next_sentence函数生成二分类任务的训练样本。

In [4]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    """这个函数的目的就是在于生成二分类的任务
    输入一个句子sentence,以及其真实的下一个句子
    有一半概率, 我接下来的句子就是真实句子
    但有有一半概率我把这个真实的next_sentence丢掉, 
    直接从paragraphs里面选择一个句子来作为next_sentence
    那么我就构建好了这么一个预测模型:
        如果next_sentence确实是,那么is_next就是True
        反之is_next就是False, 所以is_next就是标签
    """
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

- 下面的函数通过调用_get_next_sentence函数从输入paragraph生成用于下一句预测的训练样本。
- 这里paragraph是句子列表，其中每个句子都是词元列表。
- 自变量max_len指定预训练期间的BERT输入序列的最大长度。

In [5]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        # 这里把每个句子
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑1个'<cls>'词元和2个'<sep>'词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        # 这里生成的tokens, segments就可以作为bert的输入了
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        # 这里是构建下一个句子预测的样本,这里可以看到放入的是tokens, segments,而不是token_a, token_b
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

演示一下如何使用

In [6]:
paragraphs = [d2l.tokenize(
    paragraph, token='word') for paragraph in paragraphs]

此时paragraphs是将整个文章彻底拆开

In [7]:
paragraphs[2][0]

['ono',
 ',',
 'an',
 'inhabitant',
 'of',
 '<unk>',
 '(',
 'says',
 'an',
 'ancient',
 'japanese',
 'legend',
 'of',
 '<unk>',
 '545',
 ')',
 ',',
 'spent',
 'the',
 'seasons',
 'longing',
 'for',
 'his',
 'ideal',
 'of',
 'female',
 'beauty']

In [8]:

sentences = [sentence for paragraph in paragraphs
                for sentence in paragraph]

sentences是把所有句子全部拉成一个列表，不再区分paragraph

In [9]:
sentences[12]

['st.',
 'george',
 "'s",
 'church',
 'has',
 'continued',
 'to',
 'serve',
 'as',
 'a',
 'place',
 'of',
 'worship',
 'into',
 'the',
 'modern',
 'era',
 'and',
 'has',
 'been',
 'the',
 'site',
 'of',
 'archaeological',
 'excavation',
 'since',
 'the',
 'mid',
 '@-@',
 '19th',
 'century']

In [10]:
vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])

In [11]:
for paragraph in paragraphs:
    sample = _get_nsp_data_from_paragraph(paragraph,paragraphs,vocab,64)
    break

可以看到，_get_nsp_data_from_paragraph将句子处理成了[[sentence]，[segment]]形式

In [12]:
sample[0][0][:10]

['<cls>',
 'a',
 'hereditary',
 'risk',
 'for',
 'aml',
 'appears',
 'to',
 'exist',
 '<sep>']

In [13]:
sample[0][1][30:]

[1]

### 生成遮蔽语言模型任务的数据
- 为了从BERT输入序列生成遮蔽语言模型的训练样本，我们定义了以下_replace_mlm_tokens函数。
- 在其输入中，tokens是表示BERT输入序列的词元的列表，candidate_pred_positions是不包括特殊词元的BERT输入序列的词元索引的列表（特殊词元在遮蔽语言模型任务中不被预测），以及num_mlm_preds指示预测的数量（选择15%要预测的随机词元）。在 14.8.5.1节中
- 定义遮蔽语言模型任务之后，在每个预测位置，输入可以由特殊的“掩码”词元或随机词元替换，或者保持不变。最后，该函数返回可能替换后的输入词元、发生预测的词元索引和这些预测的标签。

In [14]:
"""
BERT预训练任务中被选中的15%的词中仅80%的词是真正被Masked掉的,另外10%不变，最后10%随机换为其他词，
"""
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    """
    后面代码显示 num_mlm_preds = max(1, round(len(tokens) * 0.15)) 
    num_mlm_preds为整体token长度的15%，至少为1
    
    candidate_pred_positions是把tokens里面不是'<cls>', '<sep>'的词元索引拿出来
    
    """
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        # num_mlm_preds用于控制百分之多少的词用于替换
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        ##################################################
        # 下面这部分是按比例去生成masked_token
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        #################################################    
        #  mlm_pred_position是candidate_pred_positions中选出是索引
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    # mlm_input_tokens：token替换后的token_list, pred_positions_and_labels :待预测索引和标签
    return mlm_input_tokens, pred_positions_and_labels

In [15]:
batch_size, max_len = 512, 64
for tokens, segments, is_next in _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    print(tokens,end='\n')
    print(segments,end='\n')
    print(is_next,end='\n')
    break

['<cls>', 'a', 'hereditary', 'risk', 'for', 'aml', 'appears', 'to', 'exist', '<sep>', 'in', 'slow', 'motion', ',', 'the', 'camera', 'pans', 'across', 'the', '<unk>', 'shop', 'on', 'the', 'station', '<unk>', 'as', 'the', 'duo', 'walk', 'past', '<sep>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
False


In [16]:
candidate_pred_positions = []
# tokens是一个字符串列表
for i, token in enumerate(tokens):
    # 在遮蔽语言模型任务中不会预测特殊词元
    if token in ['<cls>', '<sep>']:
        continue
    candidate_pred_positions.append(i)
num_mlm_preds = max(1, round(len(tokens) * 0.15))

- 函数演示,和上面原始的tokens对比，可以看到mlm_input_tokens已经把部分词替换成了mask，部分词进行了随机替换
- pred_positions_and_labels储存的是你具体吧哪个位置进行了替换，替换前是啥样
  - 比如(14, 'under')就是把第14个的'under'替换成了mask

In [17]:
mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(tokens,candidate_pred_positions,num_mlm_preds,vocab)
print(mlm_input_tokens)
print(pred_positions_and_labels)

['<cls>', 'a', 'hereditary', 'risk', 'for', 'aml', 'appears', 'to', 'scrutiny', '<sep>', 'in', 'mobilised', 'motion', ',', 'the', 'camera', 'pans', 'across', 'the', 'photons', 'shop', 'on', 'the', 'station', '<mask>', '<mask>', 'the', 'duo', 'walk', 'past', '<sep>']
[(24, '<unk>'), (8, 'exist'), (19, '<unk>'), (25, 'as'), (11, 'slow')]


通过调用前述的_replace_mlm_tokens函数，以下函数将BERT输入序列（tokens）作为输入，并返回输入词元的索引、发生预测的词元索引以及这些预测的标签索引。

In [18]:
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # tokens是一个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    # 输入词元的索引、发生预测的词元索引以及这些预测的标签索引。
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

依旧是演示
- vocab[mlm_input_tokens]返回的是vocab里面每个token的索引
- pred_positions就是替换的位置
- vocab[mlm_pred_labels]是具体替换的词在vocab的索引

In [19]:
pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
pred_positions_and_labels

[(8, 'exist'), (11, 'slow'), (19, '<unk>'), (24, '<unk>'), (25, 'as')]

### 将文本转换为预训练数据集

BERT预训练定制一个Dataset类。在此之前，我们仍然需要定义辅助函数_pad_bert_inputs来将特殊的“<mask>”词元附加到输入。它的参数examples包含来自两个预训练任务的辅助函数_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的输出。

In [20]:
def _pad_bert_inputs(examples, max_len, vocab):
    # max_len是用于控制单个sentence的长度的
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    # 这一段很明显是要吧单个example里面的全部数据进行一次整合，一个example对应一个paragraph的话，要整合到sentence的级别
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

先看看example长啥样

In [21]:
examples = []
for paragraph in paragraphs:
    examples.extend(_get_nsp_data_from_paragraph(
        paragraph, paragraphs, vocab, max_len))
# 获取遮蔽语言模型任务的数据
examples = [(_get_mlm_data_from_tokens(tokens, vocab)
                + (segments, is_next))
                for tokens, segments, is_next in examples]

继续演示

In [22]:
max_num_mlm_preds = round(max_len * 0.15)
all_token_ids, all_segments, valid_lens,  = [], [], []
all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
nsp_labels = []
for (token_ids, pred_positions, mlm_pred_label_ids, segments,
        is_next) in examples:
    # 长度不够的话，用'<pad>'进行填充，当然这里只能返回填充的索引
    all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (max_len - len(token_ids)), dtype=torch.long))
    # segments，长度不够填充为0
    all_segments.append(torch.tensor(segments + [0] * (max_len - len(segments)), dtype=torch.long))
    # 句子有效长度统计
    valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
    # mask替换位置，因为规定是替换15%的token，但是具体到每个sentence，长度会不够，同样用0 补齐
    all_pred_positions.append(torch.tensor(pred_positions + [0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
    # 填充词元的预测将通过乘以0权重在损失中过滤掉
    all_mlm_weights.append(torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (max_num_mlm_preds - len(pred_positions)),dtype=torch.float32))
    
    all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
    
    nsp_labels.append(torch.tensor(is_next, dtype=torch.long))

all_token_ids储存了sentence里面的句子在vocab的索引，$'<pad>'$索引为1

In [23]:
all_token_ids[0]

tensor([    3,    11, 10252,     2,     2,  2401,  1091,    10,  2373,     4,
         1791,  1769,     7,  2401,  2247,     9,    11,     2,    28,    11,
         1329,  1261,    87,  5033,    22,     2,  1487,    51,    53,   407,
            4,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1])

In [24]:
all_segments[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [25]:
valid_lens[0]

tensor(31.)

In [26]:
all_pred_positions[0]

tensor([ 2,  3,  4, 17, 25,  0,  0,  0,  0,  0])

In [27]:
all_mlm_weights[0]

tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.])

In [28]:
all_mlm_labels[0]

tensor([10188,  1828,    20,   225,  2294,     0,     0,     0,     0,     0])

In [29]:
nsp_labels[0]

tensor(1)

看到这里，我们总算可以对目前的处理进行总结，李沐大佬把目前的所有处理数据整合成了一个example，方便后续进行计算，一个example对应一个paragraph

可以看出一个标准example是包含了_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的所有东西，包括如下东西
- _get_nsp_data_from_paragraph的返回：tokens, segments, is_next
- _get_mlm_data_from_tokens 的返回：vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]
- 也就是说包含如下信息
  - tokens:句子对
    - 注意下，本代码的定义中：tokens = ['<cls>'] + tokens_a + ['<sep>']， tokens是句子对而非vocab里面的tokens
  - segments：句子对信息
  - is_next：下一个句子训练样本
  - vocab[mlm_input_tokens]：15%选出的token对于语料库里面的索引
  - pred_positions：这15%token里面，被屏蔽或者被替换的索引位置（在tokens里面的索引）
  - vocab[mlm_pred_labels]：这被替换的是哪些词

在理解前面的内容后，_WikiTextDataset没什么好说的，不过是把前面的东西整合一遍罢了

In [30]:
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表；
        # 而输出paragraphs[i]是代表段落的句子列表，其中每个句子都是词元列表
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # 获取下一句子预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        # 获取遮蔽语言模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)
         
    # 记住这个返回的序列即可。
    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

接下来是吧Dataset，转变成train_iter即可

In [31]:
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers() # 4
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

In [32]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


tokens编码

In [33]:
tokens_X[:2]

tensor([[    3,    37,  6274,    23,  3910,    61,     5,     2,  3089,     4,
             9,     5,  1290,     2,     6,     0,  6606,   570,    61, 18804,
             8,    61,     2,     7, 13216,  1530,    20,     0,     4,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    3,   106,   467,  3196,  2057,  6077,   145,  2548,    49,  1052,
           839,  2995,  1676,  5549,  3807,     7,  1047,    78,     5,   108,
            54,     5,     2,  3312,    10,    51,    88,   982,     2,   189,
            26,     2,    35,  1379,    28,    49,   189,     2,    23,  3376,
            78,     4,    37,  4804,    17,     5,   843,  1069,    75,     7,
             5,   160,  1087,    78,     5,   108,    17,    11,     2,   574,
             4

segment编码，表示是第几个句子

In [34]:
segments_X[:2]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])

每个句子有效长度

In [35]:
valid_lens_x[:100]

tensor([29., 61., 56., 24., 57., 52., 57., 39., 57., 43., 33., 49., 34., 44.,
        64., 62., 61., 60., 33., 34., 46., 40., 37., 31., 39., 48., 59., 52.,
        50., 55., 11., 48., 64., 56., 11., 64., 50., 46., 61., 52., 51., 57.,
        59., 42., 22., 47., 46., 35., 30., 48., 49., 25., 56., 33., 42., 43.,
        29., 64., 53., 52., 34., 56., 37., 58., 22., 62., 51., 37., 42., 29.,
        25., 49., 57., 26., 35., 43., 47., 48., 62., 59., 54., 56., 50., 48.,
        46., 34., 40., 49., 58., 39., 34., 63., 21., 48., 33., 40., 20., 34.,
        27., 49.])

待预测句子的位置编码

In [36]:
pred_positions_X[100:110]

tensor([[ 4,  5, 26, 27, 40, 41, 43,  0,  0,  0],
        [ 4, 16, 17, 26, 31, 32, 35, 36, 48,  0],
        [ 9, 13, 16, 21, 33, 46, 48, 53, 58, 62],
        [ 4, 12, 19, 31, 32, 33, 35, 51,  0,  0],
        [ 5, 17, 22, 30, 34, 38, 39,  0,  0,  0],
        [ 4, 10, 21, 25, 28, 31, 43,  0,  0,  0],
        [ 4, 10, 11, 12, 18, 34, 35,  0,  0,  0],
        [ 7, 11, 19, 20, 23, 26, 30,  0,  0,  0],
        [ 1,  7, 15, 23, 27, 30,  0,  0,  0,  0],
        [ 7, 12, 15, 19,  0,  0,  0,  0,  0,  0]])

掩码矩阵初试权重

In [37]:
mlm_weights_X[:10]

tensor([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]])

掩码位置

In [38]:
mlm_Y[:10]

tensor([[ 1290,  3089,    36,    66,     0,     0,     0,     0,     0,     0],
        [  145,   843,     7,   879,    26,    78,     7,     5,   876,     0],
        [   36,     9,  1590,  1727,   467,  2039,     5,  2255,     0,     0],
        [    6,    27,     6,   451,     0,     0,     0,     0,     0,     0],
        [    5,  6740,     6,     5,  7979,    20,    26,   257,     0,     0],
        [ 1507,    43,  5669,    11,    46,   782,   605,   374,     0,     0],
        [   24,     0,    15,  3214,    21,   246,    11,  6491,    11,     0],
        [  633,    10,   197,   903,    14,     8,     0,     0,     0,     0],
        [    5,   340,    12,     0, 15878,   833,     6,  8259,   382,     0],
        [ 1291,  9040,     5,   178,     9,    19,     0,     0,     0,     0]])